In [61]:
import datacube
from datacube.storage import masking
from datacube import Datacube
from datetime import datetime
from skimage import exposure

#load requiered modules #load r 
import numpy as np
import rasterio
from rasterio import features

import xarray as xr
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame

import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

In [62]:
import glue

In [63]:
from sklearn.datasets import make_classification

In [64]:
#define fuction for loading part of a bulti band GIF 

def load_gfif_full(file_name, x=1):


    dataset = rasterio.open(file_name) #open using Raterio
    {i: dtype for i, dtype in zip(dataset.indexes, dataset.dtypes)}
    data_array = dataset.read(x) #read selected band into np array
    data_xr = xr.DataArray(data_array) #turn into Xr array for fuctionality #Now don't need xr yet
    return(data_xr) #return np array into program 

In [67]:
row_number = 1641330
             #173903

In [70]:
panda_training= pd.read_csv('/g/data/u46/users/ck9738/Datasets/ML_for_smad_green/Glueviz_datasets/aus_alps_traning_master_even.csv',
                            usecols=(2,3,4,5,6,7,8,9,10,11,12), nrows= row_number)#
# open the data we are going to use for training and testing as a dataframe
panda_training

,Band 1,Band 2,Band 3,Band 4,Band 5,Band 6,Band 7,Band 8,Band 9,Band 10,class
0,278.0,386.0,411.0,650.0,1179.0,1360.0,1549.0,1571.0,1417.0,869.0,1
1,626.0,874.0,1216.0,1526.0,1847.0,2086.0,2184.0,2352.0,3197.0,2395.0,0
2,369.0,511.0,611.0,928.0,1620.0,1848.0,2005.0,2176.0,2045.0,1184.0,1
3,220.0,321.0,294.0,531.0,1165.0,1358.0,1381.0,1524.0,1032.0,585.0,1
4,442.0,641.0,795.0,1157.0,1684.0,1854.0,1997.0,2157.0,2957.0,1905.0,1
5,667.0,949.0,1299.0,1583.0,2024.0,2209.0,2430.0,2522.0,3329.0,2504.0,0
6,422.0,558.0,730.0,1035.0,1433.0,1623.0,1599.0,1904.0,2504.0,1593.0,1
7,533.0,780.0,1056.0,1426.0,1912.0,2153.0,2378.0,2520.0,3375.0,2313.0,0
8,405.0,659.0,793.0,1280.0,2151.0,2376.0,2471.0,2768.0,3012.0,1804.0,1
9,486.0,761.0,927.0,1453.0,2256.0,2424.0,2620.0,2779.0,2965.0,1873.0,0


In [71]:
red_list = panda_training['Band 1'].values.tolist()
blue_list = panda_training['Band 2'].values.tolist()
green_list = panda_training['Band 3'].values.tolist()
rede1_list = panda_training['Band 4'].values.tolist()
rede2_list = panda_training['Band 5'].values.tolist()
rede3_list = panda_training['Band 6'].values.tolist()
nir1_list = panda_training['Band 7'].values.tolist()
nir2_list = panda_training['Band 8'].values.tolist()
swir1_list = panda_training['Band 9'].values.tolist()
swir2_list = panda_training['Band 10'].values.tolist()
#slope_list = panda_training['elevation'].values.tolist()
classification = panda_training['class'].values.tolist()
#create array of features to use in classifier and array of the classification answer
how_many = len(red_list)
how_many

features = []

for i in range(how_many):
    features.append((red_list[i],
                    blue_list[i],
                    green_list[i],
                    rede1_list[i],
                    rede2_list[i],
                    rede3_list[i],
                    nir1_list[i],
                    nir2_list[i],
                    swir1_list[i],
                    swir2_list[i]))
features
features_array = np.asarray(features)

classification_array = np.asarray(classification)

In [72]:
# smad_list = panda_training['1'].values.tolist()
# max_list = panda_training['2'].values.tolist()
# min_list = panda_training['3'].values.tolist()
# #slope_list = panda_training['elevation'].values.tolist()
# classification = panda_training['class'].values.tolist()
# #create array of features to use in classifier and array of the classification answer
# how_many = len(smad_list)
# how_many

# features = []

# for i in range(how_many):
#     features.append((smad_list[i],max_list[i],min_list[i]))
# features
# features_array = np.asarray(features)

# classification_array = np.asarray(classification)

In [73]:
features_array

array([[ 278.,  386.,  411., ..., 1571., 1417.,  869.],
       [ 626.,  874., 1216., ..., 2352., 3197., 2395.],
       [ 369.,  511.,  611., ..., 2176., 2045., 1184.],
       ...,
       [ 495.,  658.,  790., ..., 2198., 2384., 1538.],
       [ 632.,  895., 1262., ..., 2392., 3035., 2306.],
       [ 228.,  328.,  366., ..., 1291., 1008.,  585.]])

In [74]:
classification_array

array([1, 0, 1, ..., 1, 0, 1])

In [75]:
#Split the feature array and the classification answer arrays into kn chuncks for the kfold validation. here K = 4

K=10

kt1,kt2,kt3,kt4,kt5,kt6,kt7,kt8,kt9,kt10 = np.split(features_array, 10)
kc1,kc2,kc3,kc4,kc5,kc6,kc7,kc8,kc9,kc10 = np.split(classification_array, 10)

In [76]:
#define classifier

#pick your method
#classify = LinearSVC(random_state=0)
#classify = RandomForestClassifier(bootstrap=False,  random_state=0)
classify = ExtraTreesClassifier(n_estimators=250, max_depth=30, bootstrap=True, random_state=0)

In [77]:
#define classifily fuction 

def train_and_test(test_f,train_f,train_c):
    """    takes data in chunks, puts the training data back into a stack to train and putputs a 
    boolian array of the test cassification
    test_f = test data to be classified. 2D array size[sample_n, features_n]
    
    train_f = list of arrays each 2D array size[sample_n, features_n].
            each is one K chunck of the total data set, to be used to train. should equal total data - 1k.
    
    train_C = list of arrays each 1D array size[sample_n]. boolean values equal to classification answer
            each is one K chunck of the total data set, to be used to train. should equal total data - 1k.
            
    returns = 1D array size[sample_n]. boolean values equal to classification answer. size equal to test_f
    """
    train_f = np.concatenate(train_f,axis=0)
    train_c = np.concatenate(train_c,axis=0)
    classify.fit(train_f,train_c)#fit the moddle to the training data
    results=classify.predict(test_f)#run predict on test data
    importance = classify.feature_importances_
    print(importance)
    return(results,importance)


In [78]:
result_one,import_one = train_and_test(kt1,[kt2,kt3,kt4,kt5,kt6,kt7,kt8,kt9,kt10],
                                [kc2,kc3,kc4,kc5,kc6,kc7,kc8,kc9,kc10])
result_two,import_two = train_and_test(kt2,[kt1,kt3,kt4,kt5,kt6,kt7,kt8,kt9,kt10],
                                [kc1,kc3,kc4,kc5,kc6,kc7,kc8,kc9,kc10])
result_three,import_three = train_and_test(kt3,[kt2,kt1,kt4,kt5,kt6,kt7,kt8,kt9,kt10],
                                [kc2,kc1,kc4,kc5,kc6,kc7,kc8,kc9,kc10])
result_four,import_four = train_and_test(kt4,[kt2,kt3,kt1,kt5,kt6,kt7,kt8,kt9,kt10],
                                [kc2,kc3,kc1,kc5,kc6,kc7,kc8,kc9,kc10])
result_five,import_five = train_and_test(kt5,[kt2,kt3,kt4,kt1,kt6,kt7,kt8,kt9,kt10],
                                [kc2,kc3,kc4,kc1,kc6,kc7,kc8,kc9,kc10])
result_six,import_six = train_and_test(kt6,[kt2,kt3,kt4,kt5,kt1,kt7,kt8,kt9,kt10],
                                [kc2,kc3,kc4,kc5,kc1,kc7,kc8,kc9,kc10])
result_seven,import_seven = train_and_test(kt7,[kt2,kt3,kt4,kt5,kt6,kt1,kt8,kt9,kt10],
                                [kc2,kc3,kc4,kc5,kc6,kc1,kc8,kc9,kc10])
result_eight,import_eight = train_and_test(kt8,[kt2,kt3,kt4,kt5,kt6,kt7,kt1,kt9,kt10],
                                [kc2,kc3,kc4,kc5,kc6,kc7,kc1,kc9,kc10])
result_nine,import_nine = train_and_test(kt9,[kt2,kt3,kt4,kt5,kt6,kt7,kt8,kt1,kt10],
                                [kc2,kc3,kc4,kc5,kc6,kc7,kc8,kc1,kc10])
result_ten,import_ten = train_and_test(kt10,[kt2,kt3,kt4,kt5,kt6,kt7,kt8,kt9,kt1],
                                [kc2,kc3,kc4,kc5,kc6,kc7,kc8,kc9,kc1])

[0.07282555 0.11430594 0.09994339 0.10953771 0.08854278 0.06828323
 0.0617009  0.08107796 0.16281945 0.14096309]
[0.07381383 0.11352657 0.10871393 0.10775119 0.0865388  0.05553959
 0.06142969 0.08000204 0.16212964 0.15055471]
[0.08055242 0.11219788 0.09914317 0.11123065 0.09412212 0.05839349
 0.0618198  0.07964057 0.15542458 0.14747531]
[0.06269638 0.11383238 0.1090377  0.11345789 0.09223769 0.0575005
 0.06533719 0.0761123  0.16370412 0.14608385]
[0.06921223 0.11846342 0.12549026 0.11662072 0.09125386 0.05665066
 0.06176446 0.07902702 0.14085529 0.14066206]
[0.07841768 0.11969912 0.09678895 0.120264   0.09135523 0.05828461
 0.06132359 0.0777111  0.1537573  0.14239843]
[0.07640836 0.1257999  0.09496302 0.11832983 0.08354749 0.06696009
 0.05955265 0.08169131 0.14283723 0.14991011]
[0.06941929 0.11573033 0.09689975 0.11444261 0.09086875 0.06746298
 0.06729776 0.07889016 0.16155635 0.13743202]
[0.07122404 0.11685344 0.11283634 0.1207566  0.08429244 0.05408449
 0.06169049 0.07989901 0.16128

In [79]:

#compleate test for each K chunk
join_results = np.concatenate((result_one,result_two,result_three,result_four,result_five,result_six,
                               result_seven,result_eight,result_nine,result_ten),axis=0)


In [80]:
classify.get_params

<bound method BaseEstimator.get_params of ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='gini',
           max_depth=30, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=250, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)>

In [81]:
panda_training['test_result']=pd.Series(join_results, index=panda_training.index)
#add results as new column to panda data frame


In [82]:
panda_training

,Band 1,Band 2,Band 3,Band 4,Band 5,Band 6,Band 7,Band 8,Band 9,Band 10,class,test_result
0,278.0,386.0,411.0,650.0,1179.0,1360.0,1549.0,1571.0,1417.0,869.0,1,1
1,626.0,874.0,1216.0,1526.0,1847.0,2086.0,2184.0,2352.0,3197.0,2395.0,0,0
2,369.0,511.0,611.0,928.0,1620.0,1848.0,2005.0,2176.0,2045.0,1184.0,1,1
3,220.0,321.0,294.0,531.0,1165.0,1358.0,1381.0,1524.0,1032.0,585.0,1,1
4,442.0,641.0,795.0,1157.0,1684.0,1854.0,1997.0,2157.0,2957.0,1905.0,1,0
5,667.0,949.0,1299.0,1583.0,2024.0,2209.0,2430.0,2522.0,3329.0,2504.0,0,0
6,422.0,558.0,730.0,1035.0,1433.0,1623.0,1599.0,1904.0,2504.0,1593.0,1,1
7,533.0,780.0,1056.0,1426.0,1912.0,2153.0,2378.0,2520.0,3375.0,2313.0,0,0
8,405.0,659.0,793.0,1280.0,2151.0,2376.0,2471.0,2768.0,3012.0,1804.0,1,1
9,486.0,761.0,927.0,1453.0,2256.0,2424.0,2620.0,2779.0,2965.0,1873.0,0,0


In [83]:
difference = np.logical_not(classification_array==join_results)
difference = difference*1
difference

array([0, 0, 0, ..., 0, 0, 0])

In [84]:
number_correct_total = np.count_nonzero(difference)
average_correct = (number_correct_total/K)
percent_correct = (average_correct/(row_number/K))*100
print(number_correct_total, average_correct, percent_correct)

38879 3887.9 2.3687497334478747


In [85]:
row_number

1641330

# KFOLD SCORE ABOVE HERE

In [ ]:
colours_class = pd.Series(classification_array)
#colours_class = colours_class.replace(to_replace=[2,1,0],value=['blue','green','orange'])
colours_class = colours_class.replace(to_replace=[0,1],value=['red','darkseagreen',
                                                                    ])#for 5 classes
colours_class = colours_class.as_matrix()
colours_class

In [ ]:
colours_r = pd.Series(join_results)
#colours_r = colours_r.replace(to_replace=[2,1,0],value=['DarkBlue','palegreen','red'])
colours_r = colours_r.replace(to_replace=[0,1,],value=['red','darkseagreen'
                                                                    ])#for 5 classes
colours_r = colours_r.as_matrix()
colours_r
#colour_r is colours result! 

In [ ]:
panda_training.plot.scatter('t_swir1','t_rededge1',c=colours_r, figsize=(12,12), alpha=0.2)
plt.title('scatter of classification result, smad vs max green')

In [ ]:
#panda_training.plot.scatter('1','3',c=colours_r, figsize=(12,12), alpha=0.2)
#plt.title('scatter of classification result, smad vs min green')

In [ ]:
#panda_training.plot.scatter('3','2',c=colours_r, figsize=(12,12),alpha=0.2)
#plt.title('scatter of classification result, Min vs max green')

In [ ]:
#panda_training.plot.scatter('1','elevation',c=colours_r, figsize=(12,12))

In [ ]:
"""panda_training.to_csv('/g/data/u46/users/ck9738/Datasets/ML_for_smad_green/kfold_results_ncas_truthed.csv')
#save data as CSV to analyse in XL"""

In [ ]:
panda_training.plot.scatter('swir1','rededge1', c=colours_class, figsize=(12,12),alpha=0.2)
plt.title('scatter of training data values, smad vs max green')

In [ ]:
panda_training.plot.scatter('1','3',c=colours_class, figsize=(12,12),alpha=0.2)
plt.title('scatter of training data values, smad vs min green')

In [ ]:
panda_training.plot.scatter('3','2',c=colours_class, figsize=(12,12),alpha=0.2)
plt.title('scatter of training data values, min vs max green')

In [ ]:
#panda_training.plot.scatter('1','elevation',c=colours_class, figsize=(12,12))